In [1]:
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 3.6.4
IPython 6.2.1

numpy 1.14.0
scipy 0.19.1
pandas 0.22.0
matplotlib 2.1.2
statsmodels 0.8.0
sklearn 0.19.1

compiler   : MSC v.1900 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 78 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit
Git hash   : 3896ce11e3ff803864dd6294ff5e80e874b7ba3d


In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob, iglob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import time
import re
#import time

In [ ]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'identification'

In [ ]:
user_id_pattern = re.compile("(\d{4}).csv")
def extract_user_id(filename):
    return user_id_pattern.search(filename).group(1)

def data_column_names(session_length):
    names = []
    for i in range(1, session_length + 1):
        names.append("site" + str(i))
    names.append("user_id")
    return names    

def prepare_train_set(path_to_csv_files, session_length=10):
    files = iglob(os.path.join(path_to_csv_files, '*.csv'))
    # site string -> (id, frequency)
    dict = {}
    data = []
    unique_site_id = 1
    for file in tqdm(files, desc = path_to_csv_files):
        user_id = extract_user_id(file)
        with open(file, 'r') as f:
            next(f) # skip header line            
            for index, line in enumerate(f):
                site = line.rstrip("\n").rpartition(',')[-1]
                                    
                # update dictionary
                if site in dict:
                    freq = dict[site][1]
                    dict[site][1] = freq + 1        
                else: 
                    dict[site] = [unique_site_id, 1]
                    unique_site_id += 1
                site_id = dict[site][0]

                # update data
                if (index % session_length == 0):
                    row = np.zeros(session_length + 1, dtype=int)
                    row[session_length] = user_id
                    data.append(row)
                    
                data[-1][index % session_length] = site_id
    # site1, site2, ..., user_id
    data_frame = pd.DataFrame(data, columns=data_column_names(session_length))
    return (data_frame, dict)
   
#prepare_train_set(os.path.join(PATH_TO_DATA, '3users'))